# We do the same transformation to the TESTPARTIAL data and apply our prediction model to get the result.

## FORCLOSED

In [1]:
import pandas as pd
import numpy as np

df=pd.read_csv("TRAIN.csv")
N=df.shape[0]
df["NMONTHS"]
df1=pd.read_csv("TESTPARTIAL.csv")
df1["NMONTHS"]=np.nan
df1["FORCLOSED"]=np.nan
df=pd.concat([df,df1],ignore_index=True)
M=df.shape[0]
Itrain=list(range(N))
Test=list(range(N,M))



In [2]:
df["OCLTV"].fillna(1,inplace=True)
df["GAP"]=df["OLTV"]-df["OCLTV"]
df["GAP"]=df["GAP"].apply(lambda x: x>0)
df["OLTV"].fillna(0, inplace=True)
df["NUMBO"].fillna(0,inplace=True)
df["DTI"].fillna(0,inplace=True)
df["CSCOREB"].fillna(0,inplace=True)
def RelativeMonth(x):
    month=int(x/10000)
    year=int(x-10000*month)
    return((year-2000)*12+month)

df["DATE1"]=df["ORIGDATE"].apply(RelativeMonth)
df["DATE2"]=df["FSTPAY"].apply(RelativeMonth)
df["DATE3"]=df["MATDT"].apply(RelativeMonth)
df["TIMGAP1"]=df["DATE2"]-df["DATE1"]
df["TIMGAP2"]=df["DATE3"]-df["DATE2"]
PreVar=['ORIGRATE', 'ORIGUPB', 'ORIGTERM','OLTV','LOANAGE','NUMBO','DTI','NO_UNITS','DLQSTATUS','ACTPER_MO','TIMGAP1']
CatVar=['SELLER','CHNL', 'FIRSTFLAG', 'PURPOSE', 'PROP', 'NO_UNITS','OCCSTAT', 'STATE',  
          'RELMORTGIND', 'GAP']
Set=df.loc[:,PreVar].values

In [3]:
from sklearn.decomposition import PCA
def transDim(tSet,dim):
    pca=PCA(n_components=dim)
    pca.fit(tSet)
    return pca.transform(tSet)
# Set=transDim(Set,'mle')
dim=3
Set=transDim(Set,dim) #decrease the dimension to 3
data=pd.DataFrame(Set[Itrain])
nameList=[('x'+str(i)) for i in range(dim)]
data.columns=nameList
data['y']=df.loc[Itrain,'FORCLOSED'].apply(lambda x: int(x))

testData=pd.DataFrame(Set[Test])
nameList=[('x'+str(i)) for i in range(dim)]
testData.columns=nameList
testData['y']=df.loc[Test,'FORCLOSED']

testData

,x0,x1,x2,y
0,-57457.732077,-44.313637,-2.534062,NaN
1,20542.266261,-26.732748,-12.209102,NaN
2,-17457.732326,-35.995927,-17.005689,NaN
3,-2457.732547,-32.644814,-19.652261,NaN
4,92542.263858,-8.733219,-0.438913,NaN
...,...,...,...,...
71192,-70457.761772,74.741283,20.403598,NaN
71193,118542.262706,-2.271994,4.935981,NaN
71194,-71457.788830,193.086358,-4.327153,NaN
71195,-2457.733779,-31.128039,0.589359,NaN


In [4]:
from sklearn.linear_model import LogisticRegression
import patsy as ps
from sklearn.metrics import confusion_matrix

def Logit(formula,dftrain,dftest,threshold=0.5):
    Ytrain,Xtrain=ps.dmatrices(formula,dftrain)
    Ytest,Xtest=ps.dmatrices(formula,dftest)
    Ytrain=np.reshape(Ytrain, (Ytrain.shape[0],)) # logistic regression wants shape as (N,) rather than (N,1)
    Ytest=np.reshape(Ytest,(Ytest.shape[0],))
    clf = LogisticRegression().fit(Xtrain,Ytrain)
    Ypred_prob=clf.predict_proba(Xtest)
    Ypred=pd.Series(Ypred_prob[:,0]<threshold)
    Ytest=pd.Series(Ytest)

    TN=sum((Ytest==0)&(Ypred==0))
    FP=sum((Ytest==0)&(Ypred==1))
    TP=sum((Ytest==1)&(Ypred==1))
    FN=sum((Ytest==1)&(Ypred==0))

    FPR = FP/(FP+TN)
    TPR = TP/(TP+FN)
    return FPR,TPR

In [5]:
N=data.shape[0]
I=np.random.permutation(range(N))
Itest2=I[0:int(N/3)]
Itrain2=I[int(N/3):N]
dftrain=data.iloc[Itrain2]
dftest=data.iloc[Itest2]
print(dftrain.shape,dftest.shape)
print(dftrain.columns)

(133334, 4) (66666, 4)
Index(['x0', 'x1', 'x2', 'y'], dtype='object')


In [6]:
from sklearn.linear_model import LogisticRegression
import patsy as ps
from sklearn.metrics import confusion_matrix


formula="y~"+"+".join(nameList)

threshold=np.linspace(0.001,0.999,999)
bestThreshold=0
FPR0=0
TPR0=0
for i in threshold:
    FPR,TPR=Logit(formula,dftrain,dftest,i)
    if np.abs(FPR-0.5)<np.abs(FPR0-0.5):
        FPR0=FPR
        TPR0=TPR
        bestThreshold=i     

In [7]:
testData["y"]=0

In [8]:
from sklearn import metrics

threshold=bestThreshold
formula="y~"+"+".join(nameList)
Ytrain,Xtrain=ps.dmatrices(formula,data)
Ytest,Xtest=ps.dmatrices(formula,testData)
Ytrain=np.reshape(Ytrain, (Ytrain.shape[0],)) 
clf = LogisticRegression().fit(Xtrain,Ytrain)
Ypred_prob=clf.predict_proba(Xtest)
Ypred=pd.Series(Ypred_prob[:,0]<threshold)

In [10]:
Ypred

0         True
1         True
2         True
3         True
4        False
         ...  
71192    False
71193    False
71194     True
71195     True
71196     True
Length: 71197, dtype: bool

In [11]:
df.loc[Test,"FORCLOSURE"]=Ypred.values
result=["LID","FORCLOSURE"]
df.loc[Test,result].to_csv('pred.csv',index=False) 

In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt

# def factor
# positive = df[df['FORCLOSED'] == True]
# negative = df[df['FORCLOSED'] == False]
# fig, ax = plt.subplots(figsize=(5, 5))
# ax.scatter(positive['ORIGUPB'], positive['ORIGRATE'], s=30, c='b', marker='o', label='Admitted')
# ax.scatter(negative['ORIGUPB'], negative['ORIGRATE'], s=30, c='r', marker='x', label='Not Admitted')
# ax.legend()
# ax.set_xlabel('ORIGUPB')
# ax.set_ylabel('ORIGRATE')